<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/portugese_whisper_base_test_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install tensorflow

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-thxpkdj4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-thxpkdj4
  Resolved https://github.com/huggingface/transformers to commit 5d36025ca13d05151b7a0c761e90d429c4644a30
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm



In [27]:
#ds = tfds.load('huggingface:multilingual_librispeech/portuguese')

mls = load_dataset("facebook/multilingual_librispeech", "portuguese", split="test")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for facebook/multilingual_librispeech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/multilingual_librispeech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next maj

Generating train split: 0 examples [00:00, ? examples/s]

Generating train.9h split: 0 examples [00:00, ? examples/s]

Generating train.1h split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
#voxpopuli_data = load_dataset("facebook/voxpopuli", "cs", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [28]:
mls

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 871
})

In [ ]:
#voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1123
})

In [29]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [30]:
model = whisper.load_model("base")

In [31]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [32]:
#Gender=mls['gender']
Audio=mls['audio']
text=mls['chapter_id']
#Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [ ]:
#Gender=voxpopuli_data['gender']
#Audio=voxpopuli_data['audio']
#text=voxpopuli_data['normalized_text']
#Gender=pd.DataFrame(Gender)
#Audio=pd.DataFrame(Audio)
#text=pd.DataFrame(text)

In [33]:
text

,0
0,10229
1,10229
2,10229
3,10229
4,10229
...,...
866,6390
867,6390
868,6390
869,6390


In [34]:
file=pd.concat([text, Audio],axis=1)
file

,0,path,array,sampling_rate
0,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00146484375, 0.00140380859375, 0.0017089843...",16000
1,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.074462890625, -0.067108154296875, -0.05853...",16000
2,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000701904296875, -0.000823974609375, -0.00...",16000
3,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[0.04034423828125, 0.0301513671875, 0.01745605...",16000
4,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000732421875, 0.000732421875, 0.0011291503...",16000
...,...,...,...,...
866,6390,/root/.cache/huggingface/datasets/downloads/ex...,"[3.0517578125e-05, 3.0517578125e-05, 3.0517578...",16000
867,6390,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.0, 0.0, -3.0517578125e-05, 0.0, 0.0, 0...",16000
868,6390,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.0, 3.0517578125e-05, 0.0, 0.0, 3.05175...",16000
869,6390,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.0, 0.0, 3.0517578125e-05, -3.051757812...",16000


In [35]:
#Gender.columns=['Gender']
text.columns=['Transcript']

In [36]:
file=pd.concat([ text, Audio],axis=1)
file

,Transcript,path,array,sampling_rate
0,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00146484375, 0.00140380859375, 0.0017089843...",16000
1,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.074462890625, -0.067108154296875, -0.05853...",16000
2,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000701904296875, -0.000823974609375, -0.00...",16000
3,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[0.04034423828125, 0.0301513671875, 0.01745605...",16000
4,10229,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000732421875, 0.000732421875, 0.0011291503...",16000
...,...,...,...,...
866,6390,/root/.cache/huggingface/datasets/downloads/ex...,"[3.0517578125e-05, 3.0517578125e-05, 3.0517578...",16000
867,6390,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.0, 0.0, -3.0517578125e-05, 0.0, 0.0, 0...",16000
868,6390,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.0, 3.0517578125e-05, 0.0, 0.0, 3.05175...",16000
869,6390,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.0, 0.0, 3.0517578125e-05, -3.051757812...",16000


In [ ]:
#male_samples=file.loc[file['Gender'] == 'male']
#male_samples

NameError: name 'male_samples' is not defined

In [ ]:
#female_samples=file.loc[file['Gender'] == 'female']
#female_samples

,Gender,Transcript,path,array,sampling_rate
8,female,nejhorší situace je v somálsku súdánu mauretán...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, -0.001922607421875, -0.00...",16000
9,female,v letecké i železniční dopravě vize nula platí...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001800537109375, 0.0030517578125, 0.0003356...",16000
10,female,jen tak potvrdíte že před dvaceti lety lidé ne...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, 0.0009765625, -0.00057983...",16000
11,female,v této souvislosti bych očekávala že alespoň d...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000518798828125, -0.0020751953125, -0.0010...",16000
13,female,já sama pocházím z regionu severní moravy ze s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000457763671875, 0.00201416015625, 0.00283...",16000
...,...,...,...,...,...
1106,female,díky tomu se mnoho informací dostává k adresát...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000244140625, -9.1552734375e-05, 0.0009155...",16000
1109,female,podle dostupných informací turecký premiér dal...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00506591796875, -0.008453369140625, -0.003...",16000
1114,female,veřejné konzultace jsou běžně zveřejňovány pře...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001251220703125, 0.00177001953125, -0.00122...",16000
1117,female,liší se rozsah průvodních dokumentů opět vesmě...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00396728515625, 0.003875732421875, 0.005584...",16000


In [37]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="portuguese")
    return result["text"]

In [38]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [39]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [40]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0
#chapter_id = normalized_text

for sample in mls:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)


  # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")




/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 1 files, WER: 0.22727272727272727, Precision: 0.8, Recall: 0.7619047619047619, F1: 0.7804878048780488


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 2 files, WER: 0.38095238095238093, Precision: 0.6388888888888888, Recall: 0.6764705882352942, F1: 0.6571428571428571


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Processed 869 files, WER: 0.3333333333333333, Precision: 0.6956521739130435, Recall: 0.6956521739130435, F1: 0.6956521739130435


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 870 files, WER: 0.4722222222222222, Precision: 0.5806451612903226, Recall: 0.5625, F1: 0.5714285714285715


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 871 files, WER: 0.3333333333333333, Precision: 0.64, Recall: 0.6666666666666666, F1: 0.6530612244897959
Average WER: 0.4160872535146157
Average Precision: 0.6029077568526267
Average Recall: 0.6177821589838497
Average F1-Score: 0.6098354649585428


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 1 files, WER: 0.7222222222222222


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 2 files, WER: 0.5555555555555556


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Processed 372 files, WER: 0.5217391304347826


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 373 files, WER: 0.7894736842105263


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 374 files, WER: 0.5757575757575758
Average WER: 0.666263036157873
